In [3]:
import gradio as gr
import google.generativeai as genai
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json
import os
from datetime import datetime
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')

class EcommerceChatbot:
    def __init__(self):
        genai.configure(api_key="API-KEY")  # Replace with your API key
        self.model = genai.GenerativeModel('gemini-2.0-flash')
        
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        
        self.index = None
        self.documents = []
        
        self.load_knowledge_base()
        
        self.chat_history = []
    
    def load_knowledge_base(self):
        """Load and index the e-commerce knowledge base"""
        
        knowledge_base = [
            "Our return policy allows returns within 30 days of purchase with original receipt and packaging.",
            "Free shipping is available on orders over $50 within the continental US.",
            "We accept Visa, MasterCard, American Express, PayPal, and Apple Pay as payment methods.",
            "Standard shipping takes 3-5 business days, expedited shipping takes 1-2 business days.",
            "To track your order, use the tracking number sent to your email after shipment.",
            "We offer 24/7 customer support via chat, email (support@store.com), and phone (1-800-123-4567).",
            "Our electronics come with a 1-year manufacturer warranty covering defects.",
            "Size exchanges are free within 14 days, just contact customer service.",
            "We price match competitor prices with valid proof within 7 days of purchase.",
            "Gift cards are available in denominations of $25, $50, $100, and $200.",
            "Account passwords can be reset using the 'Forgot Password' link on the login page.",
            "We have a loyalty program offering 5% cashback on purchases after spending $500.",
            "Damaged items can be returned for full refund or exchange with photos of damage.",
            "International shipping is available to 50+ countries with duties paid by customer.",
            "Our mobile app offers exclusive deals and faster checkout with saved payment methods.",
            "Product reviews can be submitted after purchase and help other customers make decisions.",
            "Bulk orders (50+ items) qualify for wholesale pricing with 15-20% discounts.",
            "We offer installation services for appliances and electronics for an additional fee.",
            "Seasonal sales occur during Black Friday, Cyber Monday, and end-of-season clearances.",
            "Customer data is protected with SSL encryption and we never share personal information."
        ]
        
        self.documents = knowledge_base
        
        embeddings = self.encoder.encode(knowledge_base)
        
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)  # Inner product for similarity
        
        # Normalize embeddings for cosine similarity
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings.astype('float32'))
    
    def retrieve_relevant_docs(self, query, k=3):
        """Retrieve most relevant documents for the query"""
        query_embedding = self.encoder.encode([query])
        faiss.normalize_L2(query_embedding)
        
        scores, indices = self.index.search(query_embedding.astype('float32'), k)
        
        relevant_docs = []
        for i, idx in enumerate(indices[0]):
            if scores[0][i] > 0.3:  # Similarity threshold
                relevant_docs.append({
                    'content': self.documents[idx],
                    'score': float(scores[0][i])
                })
        
        return relevant_docs
    
    def generate_response(self, user_query):
        """Generate response using RAG with Gemini"""
        
        relevant_docs = self.retrieve_relevant_docs(user_query, k=3)
        
        # Prepare context
        context = "\n".join([doc['content'] for doc in relevant_docs])
        
        prompt = f"""
        You are a helpful customer support assistant for an e-commerce store. 
        Use the following knowledge base information to answer the customer's question.
        Be friendly, concise, and helpful. If you don't know something, suggest contacting customer service.
        
        Knowledge Base:
        {context}
        
        Customer Question: {user_query}
        
        Response:
        """
        
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"I apologize, but I'm having trouble processing your request right now. Please contact our customer service at support@store.com or call 1-800-123-4567 for immediate assistance."
    
    def chat_interface(self, message, history):
        """Main chat interface function for Gradio"""
        
        # Generate response
        bot_response = self.generate_response(message)
        
        # Add to chat history
        self.chat_history.append({
            'timestamp': datetime.now().isoformat(),
            'user': message,
            'bot': bot_response
        })
        
        # Update history for Gradio
        history.append([message, bot_response])
        
        return history, ""
    
    def export_chat_history(self):
        """Export chat history to CSV"""
        if self.chat_history:
            df = pd.DataFrame(self.chat_history)
            filename = f"chat_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            df.to_csv(filename, index=False)
            return f"Chat history exported to {filename}"
        return "No chat history to export"
    
    def get_sample_questions(self):
        """Return sample questions for users"""
        return [
            "What is your return policy?",
            "How long does shipping take?",
            "What payment methods do you accept?",
            "How can I track my order?",
            "Do you offer price matching?",
            "What's your warranty policy?",
            "How do I reset my password?",
            "Do you have a loyalty program?"
        ]

def create_gradio_interface():
    """Create and return Gradio interface"""
    
    # Initialize chatbot
    chatbot = EcommerceChatbot()
    
    # Custom CSS for better styling
    css = """
    .gradio-container {
        max-width: 800px !important;
        margin: auto !important;
    }
    .chat-message {
        padding: 10px !important;
        margin: 5px !important;
        border-radius: 10px !important;
    }
    """
    
    # Create Gradio interface
    with gr.Blocks(css=css, title="E-commerce Customer Support") as demo:
        
        gr.Markdown("""
        # 🛍️ Customer Support Chatbot
        
        Welcome to our intelligent customer support! Ask me anything about:
        - Returns & Exchanges
        - Shipping & Delivery  
        - Payment Methods
        - Order Tracking
        - Warranties & Policies
        - Account Management
        """)
        
        # Main chat interface
        with gr.Row():
            with gr.Column(scale=4):
                chatbot_ui = gr.Chatbot(
                    height=400,
                    placeholder="👋 Hello! How can I help you today?"
                )
                
                with gr.Row():
                    msg_input = gr.Textbox(
                        placeholder="Type your question here...",
                        scale=4,
                        container=False
                    )
                    send_btn = gr.Button("Send", variant="primary", scale=1)
                
                clear_btn = gr.Button("Clear Chat", variant="secondary")
            
            with gr.Column(scale=1):
                gr.Markdown("### Sample Questions")
                
                sample_questions = chatbot.get_sample_questions()
                for question in sample_questions:
                    gr.Button(
                        question,
                        size="sm"
                    ).click(
                        lambda q=question: q,
                        outputs=msg_input
                    )
        
        with gr.Row():
            export_btn = gr.Button("📥 Export Chat History")
            export_status = gr.Textbox(label="Export Status", interactive=False)
        
        # Event handlers
        def respond(message, history):
            return chatbot.chat_interface(message, history)
        
        def clear_chat():
            chatbot.chat_history = []
            return []
        
        send_btn.click(
            respond,
            inputs=[msg_input, chatbot_ui],
            outputs=[chatbot_ui, msg_input]
        )
        
        msg_input.submit(
            respond,
            inputs=[msg_input, chatbot_ui],
            outputs=[chatbot_ui, msg_input]
        )
        
        clear_btn.click(
            clear_chat,
            outputs=chatbot_ui
        )
        
        export_btn.click(
            chatbot.export_chat_history,
            outputs=export_status
        )
        
        gr.Markdown("""
        ---
        **Need human assistance?** 
        📧 Email: support@store.com | 📞 Phone: 1-800-123-4567 | 💬 Live Chat: Available 24/7
        """)
    
    return demo

if __name__ == "__main__":
    demo = create_gradio_interface()
    
    demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
